In [ ]:
# Working with Small Datasets

### Data Augmentation


In [3]:
import tensorflow as tf, keras
print("TF:", tf.__version__)
print("Keras:", keras.__version__)

TF: 2.19.0
Keras: 3.10.0


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator


# creates a data generator object that transforms images
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# pick an image to transform
test_ima  = train_images[20]
img = image.img_to_array(test_ima) # convert image to numpy array
img = img.reshape((1,) + img.shape) # reshape image

i = 0

# this loop runs forever until we break, saving images to current directory with specified prefix
for batch in datagen.flow(img, save_prefix='test', save_format='jpeg'):
    plt.figure(i)
    plot = plt.imshow(image.img_to_array(batch[0]))
    i +=1
    if i > 4: # show 4 images
        break
plt.show()

ImportError: cannot import name 'ImageDataGenerator' from 'keras.preprocessing.image' (/opt/anaconda3/lib/python3.12/site-packages/keras/preprocessing/image/__init__.py)

### Pretrained Models
**We know that CNN's alone (with no dense layers) don't do anything other than map the presence of features from our input. This means we can use a pretrained CNN, one trained on millions of images, as the start of our model. This will allow us to have a very good convolutional base before adding our own dense layered classifier at the end.**


# Using a Pretrained Model


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
keras = tf.keras

# Dataset
### We will load the cat_vs_dogs datasets from the module tensorflow_datasets.

In [ ]:
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

# split the data manually into 8% training, 10% testing, 10% validation
(raw_train, raw_validation, raw_test), metadata = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True
)


In [ ]:
# create a function object that we can use to get labels
get_label_name = metadata.features['label'].int2str

# display 2 images from the dataset
for image, label in raw_train.take(5):
    plt.figure()
    plt.imshow(image)
    plt.title(get_label_name(label))

# Data Preprocessing
### Since the size of the images are all different, we need to convert them all to the same size -- create a function.

In [ ]:
IMG_SIZE = 160  # All images will be resized to 160x160


def format_example(image, label):
    # returns an image that is reshaped to IMG_SIZE

    image = tf.cast(image, tf.float32)
    image = (image / 127.5) - 1
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    return image, label


In [ ]:
train = raw_train.map(format_example)  # apply this function to all images using map()
validation = raw_validation.map(format_example)
test = raw_test.map(format_example)

In [ ]:
# have a look at our images
for image, label in train.take(2):
    plt.figure()
    plt.imshow(image)
    plt.title(get_label_name(label))

In [ ]:
# shuffle and batch the images
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 1000

train_batches = train.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
validation_batches = validation.batch(BATCH_SIZE)
test_batches = test.batch(BATCH_SIZE)

In [ ]:
# let's have a look at new images!!!
for img, label in raw_train.take(2):
    print("Original image shape: ", img.shape)
for img, label in train.take(2):
    print("New image shape: ", img.shape)
    

# Picking a Pretrained Model

In [ ]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

# create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [ ]:
base_model.summary()

In [ ]:
for image, _ in train.batches.take(1):
    pass

feature_batch = base_model(image)
print(feature_batch.shape)
# result: (32, 5, 5, 1280)

# Freezing the Base
### It simply means we won't make any changes to the weights of any layers that are frozen during training.

In [ ]:
base_model.trainable = False

In [ ]:
base_model.summary()

# Adding our Classifier

In [ ]:
# use a global average pooling layer that will average the entire 5x5 area of each 2D feature map and return to us a single 1280 element vector per filter.
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

In [ ]:
# add the prediction layer that will be a single dense neuron.
prediction_layer = keras.layers.Dense(1)

In [ ]:
# combine these layers together in a model
model = tf.keras.Sequential([
    base_model,
    global_average_layer,
    prediction_layer
])

In [ ]:
model.summary()

 # Training the Model

### Now we will train and compile the model. We will use a very small learning rate to ensure that the model does not have any major changers made to it.

In [ ]:
base_learning_rate = 0.001
model.compile(
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate),
    loss=tf.keras.losses
    # we use two classes
    .BinaryCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [ ]:
# evaluate the model right now to see how it does before training it on our new images
initial_epochs = 3
validation_steps = 20
loss0, acc0 = model.evaluate(validation_batches, steps=validation_steps)

In [ ]:
# now train it on our images
history = model.fit(train_batches, epochs=initial_epochs, validation_data=validation_batches)

acc = history.history['accuracy']
print(acc)

In [ ]:
model.save("dogs_vs_cats.keras")  # save the model and reload it at anytime in the future
new_model = tf.keras.models.load_model("dogs_vs_cats.keras")
